# Homework 1: Competitive Grammar Writing

## 1. Tagged the allowed words

In this step, we tag all the words present in allowed_words.txt with self generated tags.

##  2. Replaced all the word tags in devset.trees

The second step replaces the default given tag in devset.trees with self generated tags.

## 3. Generated grammar rules

We applied 3 different approaches to generate the grammar files.

## 4. Tested grammar files

We tested the S1.gr, S2.gr and Vocab.gr on one sentence, example_sentences.txt, and devset.txt

## 1. Tagged all allowed_words manually combined with nltk

We did not use <b>NLTK</b> directly to tag all the words because NLTK tags were giving us inconsistent grammar rules and were not in accordance to the tags given in original <b>Vocab.gr</b>.

In [ ]:
"""Mapping dictionary from NLTK tags to Vocab tags"""
pos_mappings = {"PRP": "PRP",
        "NNS": "Nouns",
        "RB": "Adv",
        "WP$": "Wh_prop",
        "MD": "MD",
        "CD": "Number",
        ")": "Punc",
        "(": "Punc",
        ":": "Punc",
        "``": "Punc",
        "''": "Punc",
        "\"": "Punc",
        ".": "Pause",
        ",": "Punc",
        "TO": "TO",
        "EX": "Adv",
        "PRP$": "Poss_pron",
        "VB": "VB",
        "CC": "CC",
        "VBP": "VBP",
        "JJS": "AdjS",
        "RBR": "RBR",
        "VBN": "VBN",
        "JJR": "Adjc",
        "IN": ["CONJ", "Prep"],
        "VBG": "VBG",
        "VBD": "VBD",
        "NN": "Noun",
        "UH": "UH",
        "POS": "POS",
        "WP": "Wh_pro",
        "NNP": "Proper",
        "WRB": "Qword",
        "VBZ": "VerbT",
        "JJ": "Adj",
        "DT": "Det"     
        }

The following code runs through the vocabulary file which contains the original words from <b>Vocab.gr</b> as well as the words from <b>allowed_words.txt</b> and tags words that have a <b>Misc</b> tag. This is done by using NLTK to tag words which are then mapped using the <b>pos_mappings</b> dictionary above. Improving vocabulary tags from <b>Misc</b> to a more fitting tag is likely help to generate better grammars.

In [ ]:
with open("Vocab_old.gr") as fo:
    with open("Vocab_new.gr", "w") as fn:
        for line in fo:
            line_tokens = line.strip().split("\t")
            if len(line_tokens) == 3 and line_tokens[1] == "Misc":
                nltk_tag = nltk.tag.pos_tag([line_tokens[2]])
                real_tag = pos_mappings.get(nltk_tag[0][1])
                if type(real_tag) == list:
                    # in the case of many possible tags
                    for i in real_tag:
                        multi_tag = line
                        multi_tag = multi_tag.replace("Misc", i)
                        fn.write(multi_tag)
                else:
                    line = line.replace("Misc", real_tag)
                    fn.write(line)
            else:
                fn.write(line)

## 2. Replaced all the word tags in devset.trees
From our observations, we decided to create our own tags for the grammar to create better and more detailed grammar.

In [ ]:
from pcfg_parse_gen import Pcfg, PcfgGenerator, CkyParse
import nltk
from nltk import Tree
from collections import Counter
import re

def print_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.pretty_print()

def draw_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.draw()

The following function takes the a <b>tree</b> file and parses it into a string that can be understood by the nltk <b>fromstring</b> method. The second task done by this function is to extract the start tags

In [ ]:
def parse_trees(filenames):
    all_trees = list()
    new_tree=" "
    start_symbol_tree=list() #used for getting the first symbol that S1 points to
    
    for filename in filenames:
        with open(filename, 'r') as f:
            lines=f.readlines()


        for line in lines:
            if line.startswith(' '):
                new_tree = new_tree + str(line).strip()
            else:
                all_trees.append(new_tree.strip())
                new_tree = " "
                new_tree = new_tree + str(line).strip()

                start_symbol_tree.append(line)

    parsed_trees=all_trees[1:]        
            
    return parsed_trees, start_symbol_tree

The <b>check_parentheses</b> and <b>find_parentheses</b> functions are mainly used to find leaf nodes of the tree so we can substitute the tags with our tags. These functions were important as we couldn't find or are unaware of existing python functions. The function was obtained from the following source: https://scipython.com/blog/parenthesis-matching-in-python/ 

In [ ]:
def check_parentheses(s):
    """ Return True if the parentheses in string s match, otherwise False. """
    j = 0
    for c in s:
        if c == ')':
            j -= 1
            if j < 0:
                return False
        elif c == '(':
            j += 1
    return j == 0

def find_parentheses(s):
    """ Find and return the location of the matching parentheses pairs in s.

    Given a string, s, return a dictionary of start: end pairs giving the
    indexes of the matching parentheses in s. Suitable exceptions are
    raised if s contains unbalanced parentheses.

    """

    # The indexes of the open parentheses are stored in a stack, implemented
    # as a list

    stack = []
    parentheses_locs = {}
    for i, c in enumerate(s):
        if c == '(':
            stack.append(i)
        elif c == ')':
            try:
                parentheses_locs[stack.pop()] = i
            except IndexError:
                raise IndexError('Too many close parentheses at index {}'
                                                                .format(i))
    if stack:
        raise IndexError('No matching close parenthesis to open parenthesis '
                         'at index {}'.format(stack.pop()))
    return parentheses_locs

def convert_locs(parentheses_locs):
    return sorted([(k,v) for k, v in parentheses_locs.items()])

In [2]:
with open("Vocab.gr","r") as f:
    vocab = f.readlines()

The following code creates a dictionary of words as key and its tag as value. This will be used later to cross check with <b>allowed_words</b> and generate our own tags.

In [ ]:
stripped_vocab=[]
tags=dict()

for v in vocab:
    if str(v).startswith("#") or str(v).startswith("\n"):
        continue
    else:
        word=""
        tag=""
        try:
            tag = str(v).split("\t")[1].strip()
            word = str(v).split("\t")[2].strip()
        except:
            print(v)
            continue
        tags[word]=tag
        stripped_vocab.append(word)

tags["."]="Punc"
tags["do"]="VB"
tags["does"]="VB"
tags["is"]="VBP"

In [ ]:
vocab,starts_sym=parse_trees(["devset.trees"])

The below code actual conversion of the old tags with our tags using the dictionary and the functions in the above cells.

In [ ]:
new_vocab=[]
for v in vocab:
    if not check_parentheses(v):
        print(v)
    parentheses_locs = find_parentheses(v)
    locs=convert_locs(parentheses_locs)
    
    old_words=[]
    new_words=[]
    
    for loc in locs:
        tag_word_list=v[loc[0]+1:loc[1]].split(" ")
        
        if len(tag_word_list) != 2:
            continue
        else:
            old_words.append(tag_word_list[0]+" "+ tag_word_list[1])
            tag_word_list[0]=tags[tag_word_list[1]] #replace old tags with our tags
            new_words.append(tag_word_list[0]+" "+ tag_word_list[1])
            
            
    for num in range(0,len(old_words)):
        v = v.replace(old_words[num],new_words[num])
    
    new_vocab.append(v)


## 3. Generated grammar rules

Three different approaches used to create the grammars:

### 3.1 Generated grammar rules based on devset.trees
We parsed the trees in the devset.trees file. After that we extracted all the grammar rules from the parsed trees.

### 3.2 Manually generated grammar rules
Some of the rules were generated manually in order to bring correctness into our grammar.

### 3.3 Generated self rules for S2
S2 rules were created through code which covers how often a word follows another word in devset.txt

The <b>generate_grammer</b> function uses the output from the parse_tree function to generate the new grammar, it outputs the grammar in <b>Chomsky Normal Form</b> along with weights for each rule. We also obtain weights for the terminal nodes which is used to generate Vocab.gr

In [ ]:
def generate_grammers(trees):
    #generates non-terminal (s1) and terminal (vocab) grammer
    
    grammar=dict()
    
    for tree in trees:
        t = Tree.fromstring(tree)
        t.chomsky_normal_form()
        productions = t.productions()
        for production in productions:
            has_quotes = str(production).find("\'")
            if has_quotes == -1:
                if production in grammar:
                    grammar[production] += 1
                else:
                    grammar[production] = 1

    return grammar

The generate_start_symbols functions creates the initial rules <b>(of the type S1 -> tags)</b>

In [ ]:
def generate_start_symbols(start_symb):
    start_symb_dict=dict()
    symbol_list=list()

    for one_line in start_symb:
        symbol_list.append(str(one_line).split("(")[1].strip())

    start_symb_dict=Counter(symbol_list)
    return start_symb_dict


The below function is used to generate tags for <b>Vocab.gr</b> from the devset trees. After that we check whether those words are present in <b>allowed_words</b> and add them to the <b>Vocab.gr</b>.

In [ ]:
def generate_vocab(vocab):
    F=open('allowed_words.txt','r')
    wordsList = []
    for line in F:
        wordsList.append(line.strip('\n'))
    tags=nltk.tag.pos_tag(wordsList) #Creating tags for words in allowed_words.txt
    vocabFile=open('Vocab.gr','r')
    vocabList = []
    for line in vocabFile:
        vocabList.append(line.strip('\n'))
    present_words=dict()
    for i in vocabList: #Creating dictionary present_words of type (word,tag):weight in vocab.gr file
        if(i.startswith('1')):
            x=i.split(' ')
            elements= list(filter(None,x))
            present_words[(elements[2],elements[1])]=1
    new_words=dict(tags) #Creating dictionary new_words of type (word,tag):weight for allowed_words.txt file
    for key in new_words:
        present_words[(key,new_words[key])]=1 #Including new_words dictionary elements in present_words dictionary
    wordsDevset=dict()
    for key in vocab.keys():
        word=str(key).split(' ')[-1]
        a=word.strip("\'")
        wordsDevset[(a,str(key).split(' ')[0])]=vocab[key] #Creating dictionary wordsDevset of type (word,tag):weight for devset.txt file
    difference=set(dict(wordsDevset.keys()).keys())-(set(dict(present_words.keys()).keys())) #Excluding words from devset that are not in allowed_words.txt

    wordsDevset={key: wordsDevset[key] for key in wordsDevset if key[0] not in difference}

    for key in wordsDevset:
        present_words[key]=wordsDevset[key] #Including wordsDevset dictionary elements in present_words dictionary

    present_words={key: present_words[key] for key in present_words if key[0] != ["Misc","Noun","Proper","VerbT"]}
    
    for key in present_words: #Writing the new Vocab file
        cfg=str(present_words[key])+"\t"+key[1]+"\t"+key[0]
        with open('Vocab.gr', 'a') as f:
           f.write(cfg)
           f.write("\n")
    return

The <b>write_ss</b> and <b>generate_s1</b> are functions to write the generated grammars to <b>S1.gr</b> and <b>S2.gr</b>

In [ ]:
def generate_s1(grammar):
    for key, value in grammar.items():

        cfg=""

        split_key = str(key).split("->")

        split_non_terminal=split_key[1].split()

        try:
            cfg=str(value)+"\t"+str(split_key[0]).strip()+"\t"+str(split_non_terminal[0]).strip()+" "+str(split_non_terminal[1]).strip()
            
            with open('S2.gr', 'a') as f:
                f.write(cfg)
                f.write("\n") 
        except:
            cfg=str(value)+"\t"+str(split_key[0]).strip()+"\t"+str(split_non_terminal[0]).strip()
            with open('S2.gr', 'a') as f:
                f.write(cfg)
                f.write("\n") 
            

def write_ss(ss):
    for key, value in ss.items(): 
        to_write=str(value)+"\tS2\t"+str(key)
        with open('S2.gr', 'a') as f:
                f.write(to_write)
                f.write("\n")
    return

The following code is a function to create the <b>S2</b> rules and takes as input the S2 sentences along with dictionary of all the words present in <b>Vocab.gr</b>. It appends the S2 grammar with the new rule generated. It generates general rules of a word following another for back-off .

In [ ]:
def create_S2_rules(sent,present_words):
    sentence=sent.rstrip()
    words=sentence.split(" ")
    words.reverse()
    print(words)
    for i in range(len(words)):
        print(words[i],present_words[words[i]])
        with open('S2.gr', 'a') as f:
            if(i==len(words)-1):
                line="1   "+"S2 "+present_words[words[i]]+" _"+present_words[words[i-1]]
            elif(i==0):
                line='1   _'+present_words[words[i]]+" "+present_words[words[i]]
            else:
                line='1   _'+present_words[words[i]]+" "+present_words[words[i]]+" _"+present_words[words[i-1]]
            f.write(line)
            f.write("\n")

The below code reads the Vocab file and creates a dictionary of form <b>(word,tag)</b>. This dictionary along with sentences from devset are passed to the <b>create_S2_rules</b> function.

In [ ]:
vocabFile=open('Vocab.gr','r')
vocabList = []
for line in vocabFile:
    vocabList.append(line.strip('\n'))
present_words=dict()
for i in vocabList: #Creating dictionary present_words of type (word,tag):weight in vocab.gr file
    if(i.startswith('1')):
        x=i.split(' ')
        elements= list(filter(None,x))
        present_words[elements[2]]=elements[1]
with open("devset.txt") as f:
    for line in f:
        print(line)
        create_S2_rules(line,present_words) 

In [ ]:
new_vocab[2]

In [ ]:
s1=generate_grammers(new_vocab)
generate_s1(s1)

In [ ]:
ss=generate_start_symbols(starts_sym)

write_ss(ss)


## 4. Tested the grammar files

### 4.1 Tested on a sentences

In [6]:
parse_gram = Pcfg(["S1.gr","S2.gr","Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.00001)
ce, trees = parser.parse_sentences(["Arthur suggested that the castle be carried ."])
print("-cross entropy: {}".format(ce))
for tree_string in trees:
    print_tree(tree_string)

(TOP (S1 (VBN_Det (_VBN (Proper Arthur) (VBN suggested)) (_Det (Det that) (NP_VBD (NP (NP (Det the) (Noun castle)) (VP (VB be) )) (_VBD (VBD carried) (Punc .))))) ) )
-cross entropy: -10.600491674906678
                             TOP                                      
                              |                                        
                              S1                                      
                              |                                        
                           VBN_Det                                    
         _____________________|__________                              
        |                               _Det                          
        |               _________________|______                       
        |              |                      NP_VBD                  
        |              |                  ______|_____________         
        |              |                 NP                   |       
        |  

#reading grammar file: S1.gr
#reading grammar file: S2.gr
#Ignored cycle NP -> NP
#Ignored cycle NP -> NP
#reading grammar file: Vocab.gr
#parsing: ['Arthur', 'suggested', 'that', 'the', 'castle', 'be', 'carried', '.']
#-cross entropy (bits/word): -10.6005


### 4.2 Tested on example_sentences.txt

In [4]:
parse_gram = Pcfg(["S1.gr","S2.gr","Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.00001)
ce, trees = parser.parse_file('example_sentences.txt')
print("-cross entropy: {}".format(ce))

#reading grammar file: S1.gr
#reading grammar file: S2.gr
#Ignored cycle NP -> NP
#Ignored cycle NP -> NP
#reading grammar file: Vocab.gr
#parsing: ['Arthur', 'is', 'the', 'king', '.']
#parsing: ['Arthur', 'rides', 'the', 'horse', 'near', 'the', 'castle', '.']
#parsing: ['riding', 'to', 'Camelot', 'is', 'hard', '.']
#parsing: ['do', 'coconuts', 'speak', '?']
#parsing: ['what', 'does', 'Arthur', 'ride', '?']
#parsing: ['who', 'does', 'Arthur', 'suggest', 'she', 'carry', '?']
#parsing: ['why', 'does', 'England', 'have', 'a', 'king', '?']
#parsing: ['are', 'they', 'suggesting', 'Arthur', 'ride', 'to', 'Camelot', '?']


(TOP (S2 (S (NP (Proper Arthur) ) (S|<VP-Punc> (VP (VBP is) (NP (Det the) (Noun king))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Arthur) ) (S|<VP-Punc> (VP (VerbT rides) (NP (NP (Det the) (Noun horse)) (PP (Prep near) (NP (Det the) (Noun castle))))) (Punc .))) ) )
(TOP (S1 (VP (VBG riding) (PP (TO to) (NP (NNP Camelot) ))) (_VP (VP (VBP is) (ADJP (Adj hard) )) (Punc .))) )
(TOP (S1 (Qst (_Aux (Aux do) (_Nouns (Nouns coconuts) (VB speak))) ) (Qmark ?)) )
(TOP (S1 (Qst (Qword what) (_Aux (Aux does) (_Proper (Proper Arthur) (VBP ride)))) (Qmark ?)) )
(TOP (S2 (SBARQ (WHNP (Wh_pro who) ) (SBARQ|<SQ-Qmark> (SQ (VB does) (SQ|<NP-VP> (NP (Proper Arthur) ) (VP (VB suggest) (S (NP (PRP she) ) (VP (VB carry) ))))) (Qmark ?))) ) )
(TOP (S2 (SBARQ (WHNP (Qword why) ) (SBARQ|<SQ-Qmark> (SQ (VB does) (SQ|<NP-VP> (NP (NNP England) ) (VP (VB have) (NP (Det a) (Noun king))))) (Qmark ?))) ) )
(TOP (S1 (Qst (_VBP (VBP are) (_PRP (PRP they) (_VBG (VBG suggesting) (_Proper (Proper Arthur) (_VBP (VBP ride) (_

#parsing: ['five', 'strangers', 'are', 'at', 'the', 'Round', 'Table', '.']
#parsing: ['Guinevere', 'might', 'have', 'known', '.']
#parsing: ['Guinevere', 'should', 'be', 'riding', 'with', 'Patsy', '.']
#parsing: ['it', 'is', 'Sir', 'Lancelot', 'who', 'knows', 'Zoot', '!']
#parsing: ['either', 'Arthur', 'knows', 'or', 'Patsy', 'does', '.']
#parsing: ['neither', 'Sir', 'Lancelot', 'nor', 'Guinevere', 'will', 'speak', 'of', 'it', '.']


(TOP (S2 (S (NP (Number five) (Nouns strangers)) (S|<VP-Punc> (VP (VBP are) (PP (Prep at) (NP (Det the) (NP|<Noun-Noun> (Noun Round) (Noun Table))))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Guinevere) ) (S|<VP-Punc> (VP (MD might) (VP (VB have) (VP (VBN known) ))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Guinevere) ) (S|<VP-Punc> (VP (MD should) (VP (VB be) (VP (VBG riding) (PP (Prep with) (NP (Proper Patsy) ))))) (Punc .))) ) )
(TOP (S1 (NP (PRP it) ) (_VP (_VerbT (VerbT is) (_Proper (Proper Sir) (_Noun (Noun Lancelot) (_Wh_pro (Wh_pro who) (_VBS (VBS knows) (_Proper (Proper Zoot) )))))) (Punc !))) )
(TOP (S1 (_CC_either (CC_either either) (_CC_balance_either (_Proper (Proper Arthur) (_VBS (VBS knows) )) (_CC_end_either (CC_or or) (_Proper (Proper Patsy) (_Aux (Aux does) (Punc .)))))) ) )
(TOP (S1 (S1 (_CC_neither (CC_neither neither) (_CC_balance_neither (_Proper (Proper Sir) (_Noun (Noun Lancelot) )) (_CC_end_neither (CC_nor nor) (_Proper (Proper Guinevere) )))) ) (VP_Punc (VP (MD will

#parsing: ['the', 'Holy', 'Grail', 'was', 'covered', 'by', 'a', 'yellow', 'fruit', '.']
#parsing: ['Zoot', 'might', 'have', 'been', 'carried', 'by', 'a', 'swallow', '.']
#parsing: ['Arthur', 'rode', 'to', 'Camelot', 'and', 'drank', 'from', 'his', 'chalice', '.']
#parsing: ['they', 'migrate', 'precisely', 'because', 'they', 'know', 'they', 'will', 'grow', '.']


(TOP (S2 (S (NP (Det the) (NP|<Proper-Noun> (Proper Holy) (Noun Grail))) (S|<VP-Punc> (VP (VBD was) (VP (VBN covered) (PP (Prep by) (NP (Det a) (NP|<Adj-Noun> (Adj yellow) (Noun fruit)))))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Zoot) ) (S|<VP-Punc> (VP (MD might) (VP (VB have) (VP (VBN been) (VP (VBN carried) (PP (Prep by) (NP (Det a) (Noun swallow))))))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Arthur) ) (S|<VP-Punc> (VP (VP (VBD rode) (PP (TO to) (NP (NNP Camelot) ))) (VP|<CC-VP> (CC and) (VP (VBD drank) (PP (Prep from) (NP (Poss_pron his) (Noun chalice)))))) (Punc .))) ) )
(TOP (S1 (NP (PRP they) ) (VP (VP (VB migrate) (ADVP (ADV precisely) )) (SBAR (CONJ because) (S (NP (PRP they) ) (S|<VP-Punc> (VP (VB know) (S (NP (PRP they) ) (VP (MD will) (VP (VB grow) )))) (Punc .)))))) )
(TOP (S1 (VP (VB do) (ADVP (ADV not) )) (_VP (VP (VB speak) ) (Punc !))) )
(TOP (S2 (S (NP (Proper Arthur) ) (S|<VP-Punc> (VP (MD will) (VP (VB have) (VP (VBN been) (VP (VBG riding) (PP (Prep for) (NP (Number 

#parsing: ['do', 'not', 'speak', '!']
#parsing: ['Arthur', 'will', 'have', 'been', 'riding', 'for', 'eight', 'nights', '.']
#parsing: ['Arthur', ',', 'sixty', 'inches', ',', 'is', 'a', 'tiny', 'king', '.']
#parsing: ['Arthur', 'knows', 'Patsy', ',', 'the', 'trusty', 'servant', '.']
#parsing: ['Arthur', 'and', 'Guinevere', 'migrate', 'frequently', '.']
#parsing: ['he', 'knows', 'what', 'they', 'are', 'covering', 'with', 'that', 'story', '.']
#parsing: ['Arthur', 'suggested', 'that', 'the', 'castle', 'be', 'carried', '.']


(TOP (S2 (S (NP (NP (Proper Arthur) ) (NP|<Pause-NP-Pause> (Pause ,) (NP|<NP-Pause> (NP (Number sixty) (Nouns inches)) (Pause ,)))) (S|<VP-Punc> (VP (VBP is) (NP (Det a) (NP|<Adj-Noun> (Adj tiny) (Noun king)))) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Arthur) ) (S|<VP-Punc> (VP (VBS knows) (S1 (NP (NP (Proper Patsy) ) (NP|<Pause-NP> (Pause ,) (NP (Det the) (NP|<Adj-Noun> (Adj trusty) (Noun servant))))) )) (Punc .))) ) )
(TOP (S2 (S (NP (Proper Arthur) (CC_Proper (CC and) (Proper Guinevere))) (S|<VP-Punc> (VP (VB migrate) (ADVP (ADV frequently) )) (Punc .))) ) )
(TOP (S2 (PRP he) (_VBS (VBS knows) (_Wh_pro (Wh_pro what) (_NP (NP (PRP they) ) (VP (VBP are) (NP (NP (VBG covering) ) (PP (Prep with) (NP (Det that) (NP|<Noun-Punc> (Noun story) (Punc .)))))))))) )
(TOP (S1 (VBN_Det (_VBN (Proper Arthur) (VBN suggested)) (_Det (Det that) (NP_VBD (NP (NP (Det the) (Noun castle)) (VP (VB be) )) (_VBD (VBD carried) (Punc .))))) ) )
(TOP (S2 (S (NP (Det the) (Noun king)) (S|<VP-Punc> (VP (VBD drank)

#parsing: ['the', 'king', 'drank', 'to', 'the', 'castle', 'that', 'was', 'his', 'home', '.']
#parsing: ['when', 'the', 'king', 'drinks', ',', 'Patsy', 'drinks', '.']
#-cross entropy (bits/word): -9.53778


### Tested on devset.txt

In [3]:
parse_gram = Pcfg(["S1.gr","S2.gr","Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.00001)
ce, trees = parser.parse_file('devset.txt')
print("-cross entropy: {}".format(ce))

#reading grammar file: S1.gr
#reading grammar file: S2.gr
#Ignored cycle NP -> NP
#Ignored cycle NP -> NP
#reading grammar file: Vocab.gr
#parsing: ['Whoa', 'there', '!']
#parsing: ['Halt', '!']
#parsing: ['Who', 'goes', 'there', '?']
#parsing: ['It', 'is', 'I', ',', 'Arthur', ',', 'son', 'of', 'Uther', 'Pendragon', ',', 'from', 'the', 'castle', 'of', 'Camelot', '.']


(TOP (S2 (INTJ (Noun Whoa) (INTJ|<ADVP-Punc> (ADVP (Adv there) ) (Punc !))) ) )
(TOP (S1 (NP (Noun Halt) (Punc !)) ) )
(TOP (S2 (SBARQ (WHNP (Wh_pro Who) ) (SBARQ|<SQ-Qmark> (SQ (VP (VBS goes) (ADVP (Adv there) )) ) (Qmark ?))) ) )
(TOP (S2 (S (NP (PRP It) ) (S|<VP-Punc> (VP (VBP is) (NP (NP (NP (NP (PRP I) ) (NP|<Pause-NP-Pause-NP-Pause> (Pause ,) (NP|<NP-Pause-NP-Pause> (NP (Proper Arthur) ) (NP|<Pause-NP-Pause> (Pause ,) (NP|<NP-Pause> (NP (NP (Noun son) ) (PP (Prep of) (NP (Noun Uther) (Noun Pendragon)))) (Pause ,)))))) (PP (Prep from) (NP (Det the) (Noun castle)))) (PP (Prep of) (NP (NNP Camelot) )))) (Punc .))) ) )


#parsing: ['King', 'of', 'the', 'Britons', ',', 'defeator', 'of', 'the', 'Saxons', ',', 'sovereign', 'of', 'all', 'England', '!']


(TOP (S1 (VP (VBG King) (PP (Prep of) (NP (NP (Det the) (NNPS Britons)) (NP|<Pause-NP-Pause-ADJP-Punc> (Pause ,) (NP|<NP-Pause-ADJP-Punc> (NP (NP (Noun defeator) ) (PP (Prep of) (NP (Det the) (Nouns Saxons)))) (NP|<Pause-ADJP-Punc> (Pause ,) (NP|<ADJP-Punc> (ADJP (Noun sovereign) (PP (Prep of) (NP (Det all) (Nbar (NNP England) )))) (Punc !)))))))) ) )
(TOP (S2 (S (VP (Noun Pull) (NP (Det the) (NP|<Adj-PRP> (Adj other) (PRP one)))) (Punc !)) ) )
(TOP (S2 (S (NP (PRP I) ) (S|<VP-Punc> (VP (VBP am) ) (Punc .))) ) )
(TOP (S2 (FRAG (CC And) (FRAG|<NP-NP-Punc> (NP (Det this) ) (FRAG|<NP-Punc> (NP (Poss_pron my) (NP|<Adj-Noun-Proper> (Adj trusty) (NP|<Noun-Proper> (Noun servant) (Proper Patsy)))) (Punc .)))) ) )


#parsing: ['Pull', 'the', 'other', 'one', '!']
#parsing: ['I', 'am', '.']
#parsing: ['And', 'this', 'my', 'trusty', 'servant', 'Patsy', '.']
#parsing: ['We', 'have', 'ridden', 'the', 'length', 'and', 'breadth', 'of', 'the', 'land', 'in', 'search', 'of', 'knights', 'who', 'will', 'join', 'me', 'in', 'my', 'court', 'of', 'Camelot', '.']


...

#parsing: ['Oh', ',', 'cut', 'your', 'own', 'head', 'off', '!']
#parsing: ['Yes', ',', 'do', 'us', 'all', 'a', 'favor', '!']
#parsing: ['What', '?']
#parsing: ['Yapping', 'on', 'all', 'the', 'time', '.']
#parsing: ["You're", 'lucky', '.']
#parsing: ["You're", 'not', 'next', 'to', 'him', '.']
#parsing: ['What', 'do', 'you', 'mean', '?']
#parsing: ['You', 'snore', '.']
#parsing: ['Oh', 'I', 'do', "n't", '--', 'anyway', ',', 'you', "'ve", 'got', 'bad', 'breath', '.']


(TOP (S2 (S (ADVP (UH Oh) ) (S|<NP-VP-Punc> (NP (PRP I) ) (S|<VP-Punc> (VP (VP (VB do) (ADVP (Adv n't) )) (VP|<Pause-S> (Pause --) (S (INTJ (Adv anyway) ) (S|<Pause-NP-VP> (Pause ,) (S|<NP-VP> (NP (PRP you) ) (VP (OTH 've) (VP (VBD got) (NP (Adj bad) (Noun breath))))))))) (Punc .)))) ) )
(TOP (S2 (S (ADVP (Adv Well) ) (S|<NP-VP-Punc> (NP (NP (Poss_pron its) ) (SBAR (Adv only) (SBAR|<CONJ-S> (CONJ because) (S (NP (PRP you) ) (VP (VB do) (ADVP (Adv n't) )))))) (S|<VP-Punc> (VP (Noun brush) (NP (Poss_pron my) (Nouns teeth))) (Punc .)))) ) )
(TOP (S2 (S (ADVP (UH Oh) ) (S|<NP-VP-Punc> (NP (Noun stop) ) (S|<VP-Punc> (VP (VP (Noun bitching) ) (VP|<CC-VP> (CC and) (VP (VB let) (VP (OTH 's) (VP (VB go) (VP (VB have) (NP (Noun tea) ))))))) (Punc .)))) ) )
(TOP (S1 (NP (NP (Det All) (Noun right)) (NP|<Pause-NP-Pause-NP-Pause> (Pause ,) (NP|<NP-Pause-NP-Pause> (NP (Det all) (Noun right)) (NP|<Pause-NP-Pause> (Pause ,) (NP|<NP-Pause> (NP (Det all) (Noun right)) (Pause .)))))) ) )


#parsing: ['Well', 'its', 'only', 'because', 'you', 'do', "n't", 'brush', 'my', 'teeth', '.']
#parsing: ['Oh', 'stop', 'bitching', 'and', 'let', "'s", 'go', 'have', 'tea', '.']
#parsing: ['All', 'right', ',', 'all', 'right', ',', 'all', 'right', '.']
#parsing: ['We', "'ll", 'kill', 'him', 'first', 'and', 'then', 'have', 'tea', 'and', 'biscuits', '.']


(TOP (S2 (S (S (NP (PRP We) ) (VP (OTH 'll) (VP|<Noun-NP-ADVP> (Noun kill) (VP|<NP-ADVP> (NP (PRP him) ) (ADVP (Adv first) ))))) (S|<CC-S-Punc> (CC and) (S|<S-Punc> (S (NP (Adv then) ) (VP (VB have) (NP (NP (Noun tea) ) (NP|<CC-NP> (CC and) (NP (Nouns biscuits) ))))) (Punc .)))) ) )
(TOP (S2 (INTJ (UH Yes) (Punc .)) ) )
(TOP (S2 (UH Oh) (_Pause (Pause ,) (_CC (CC but) (_ADV (ADV not) (_Nouns (Nouns biscuits) (Punc .)))))) )
-cross entropy: -9.666825363823383


#parsing: ['Yes', '.']
#parsing: ['Oh', ',', 'but', 'not', 'biscuits', '.']
#-cross entropy (bits/word): -9.66683
